# ⭐ Tutorial: Structural Break Detection (ADF) with RiskLabAI

This notebook is a tutorial for the structural break detection methods in the `RiskLabAI` library, based on Chapter 17 of 'Advances in Financial Machine Learning' by Marcos López de Prado.

The goal is to detect explosive behavior (bubbles) in a time series. We will use the Augmented Dickey-Fuller (ADF) test, but applied in two different ways:

We will demonstrate:
1.  **Data Preparation:** Load and plot log-prices for Bitcoin (`BTC-USD`) to get a clear example of bubbles.
2.  **Expanding Window ADF:** Use the `get_expanding_window_adf` function to see how the ADF statistic evolves over time. This helps visually identify when the series enters an explosive (bubble) regime.
3.  **Backward Supremum ADF (BSADF):** Use the `get_bsadf_statistic` function to get a single test statistic for the *origination* of a bubble.
4.  **Conclusion:** Compare the results.

## 0. Setup and Imports

First, we import our libraries and the necessary modules from `RiskLabAI`.

In [ ]:
# Standard Imports
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

# RiskLabAI Imports
import RiskLabAI.features.structural_breaks as sb
import RiskLabAI.utils.publication_plots as pub_plots

# --- Notebook Configuration ---
pub_plots.setup_publication_style()

## 1. Load and Prepare Data

We'll use Bitcoin (`BTC-USD`) data from 2015 to 2023, as it contains several well-known bubble periods. We will use the log of the closing price, as required by the ADF test.

In [ ]:
# Load BTC data
btc_price = yf.Ticker("BTC-USD").history(start="2015-01-01", end="2023-01-01")['Close']

# Convert to log price
log_price = np.log(btc_price).to_frame("LogPrice")

# --- Plotting ---
fig, ax = plt.subplots(figsize=(14, 7))
log_price.plot(ax=ax, label='Log Price (BTC-USD)')

pub_plots.apply_plot_style(
    ax, 
    'Log Price of BTC-USD (2015-2023)', 
    'Date', 
    'Log Price'
)
ax.legend()
plt.show()

## 2. Expanding Window ADF Test

Now, we'll run the `get_expanding_window_adf` test. This function calculates the ADF t-statistic for an expanding window of data, starting with a `min_sample_length` (we'll use 1 year / 365 days).

A standard ADF test looks for stationarity (a *negative* t-stat). However, when testing for bubbles, we look for **explosive behavior**, which is indicated by a *positive and rising* t-statistic that crosses a critical value.

In [ ]:
print("Running expanding window ADF...")
adf_stats = sb.get_expanding_window_adf(
    log_price=log_price['LogPrice'],
    min_sample_length=365, 
    constant='c',  # 'c' for constant/intercept
    lags=1         # Number of lags for delta prices
)
print("Calculation complete.")
adf_stats.tail()

## 3. Plotting the ADF Statistic

Let's plot the log-price against the ADF statistic. We'll add a hypothetical critical value line. When the ADF statistic (blue) rises *above* this line, it signals an explosive, bubble-like regime.

In [ ]:
# Critical value (example, real values must be computed via Monte Carlo)
# 95% critical value for this test is often around 1.6 - 2.0
critical_value = 1.64 

# --- Plotting ---
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Panel 1: Log Price
ax1.plot(log_price.index, log_price['LogPrice'], label='Log Price (BTC-USD)', color='C0')
pub_plots.apply_plot_style(ax1, 'BTC-USD Log Price and ADF Statistic', '', 'Log Price')
ax1.legend(loc='upper left')

# Panel 2: ADF Statistic
ax2.plot(adf_stats.index, adf_stats, label='Expanding Window ADF Stat', color='C1')
ax2.axhline(y=critical_value, color='red', linestyle='--', linewidth=2, label=f'95% Critical Value ({critical_value})')
pub_plots.apply_plot_style(ax2, '', 'Date', 'ADF t-Statistic')
ax2.legend(loc='upper left')

plt.tight_layout()
plt.show()

**Analysis:** The plot clearly shows the power of this method. 

* During normal periods, the ADF statistic (orange) stays low.
* During the 2017-2018 bubble, the statistic rises dramatically, crossing the critical value.
* It spikes again during the 2020-2021 bubble.

This confirms the `get_expanding_window_adf` function is correctly identifying periods of explosive behavior.

## 4. Backward Supremum ADF (BSADF) Test

While the expanding window plot is great for visualization, a formal test requires a single statistic. The BSADF test (implemented as `get_bsadf_statistic`) finds the *supremum* (the highest peak) of the ADF statistics calculated over *all possible start dates*.

This single value can be compared against a critical value table to formally reject the null hypothesis of no bubble.

In [ ]:
print("Running BSADF test...")
bsadf_result = sb.get_bsadf_statistic(
    log_price=log_price,
    min_sample_length=365,
    constant='c',
    lags=1
)

print(f"BSADF Statistic: {bsadf_result['gsadf']:.4f}")
print(f"(Test run as of: {bsadf_result['Time'].date()})")

## 5. Conclusion

This notebook demonstrated the two primary functions of the `structural_breaks` module:

1.  **`get_expanding_window_adf`:** This function is excellent for visualization. It generates a time series of the ADF statistic, allowing us to see *when* the market entered and exited a bubble-like state.

2.  **`get_bsadf_statistic`:** This function is used for formal hypothesis testing. It returns the single highest ADF statistic (the supremum) from all possible windows, which can be compared against pre-computed critical values to determine if a bubble occurred.

Together, these functions provide a robust toolkit for detecting structural breaks.